In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


2023-10-29 11:09:54.352577: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 11:09:54.384002: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 11:09:54.384044: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 11:09:54.384082: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 11:09:54.390993: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 11:09:54.391899: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# Load the dataset
data = pd.read_csv("../data/data.csv") 

In [3]:
# Convert the 'Date' column to a datetime object
data['Date'] = pd.to_datetime(data['Date'])

In [5]:
# Convert the 'Date' column to separate numerical features (e.g., year, month, day)
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

# Drop the original 'Date' column
data = data.drop(columns=['Date'])

In [6]:
data

,GemstoneName,Color,Clarity,Cut,Weight,Price,Year,Month,Day
0,Spinel,Brown,Transparent,Oval,1.36,9000,2021,1,15
1,Cat's Eye,Yellow,Translucent,Oval,0.75,18000,2021,1,15
2,Spinel,Brown,Transparent,Oval,1.36,9000,2021,1,17
3,Cat's Eye,Yellow,Translucent,Oval,0.75,18000,2021,1,28
4,Sapphire,Purple,Transparent,Emerald,0.73,20000,2021,2,12
...,...,...,...,...,...,...,...,...,...
696,Ruby,Red,Transparent,Emerald,0.51,30000,2022,5,11
697,Ruby,Red,Transparent,Emerald,0.95,30000,2022,8,4
698,Ruby,Purple,Transparent,Oval,0.73,30000,2022,9,20
699,Ruby,Purple,Transparent,Oval,0.73,30000,2023,1,17


In [7]:
# Encode categorical variables (e.g., GemstoneName, Color, Clarity, Cut)
label_encoders = {}
for column in ['GemstoneName', 'Color', 'Clarity', 'Cut']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le


In [8]:
# Split the data into features and target
X = data.drop(columns=['Price'])
y = data['Price']

In [9]:
# Normalize numerical features
scaler = StandardScaler()
X[['Weight']] = scaler.fit_transform(X[['Weight']])

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
model = keras.Sequential()

# Add an input layer
model.add(keras.layers.Input(shape=(X_train.shape[1],)))

# Add hidden layers
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))

# Add the output layer
model.add(keras.layers.Dense(1, activation='linear'))


In [12]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


In [13]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50


18/18 [==============================] - 1s 17ms/step - loss: 9880010752.0000 - mae: 65647.3672 - val_loss: 9572652032.0000 - val_mae: 65291.1758
Epoch 2/50
18/18 [==============================] - 0s 2ms/step - loss: 9692542976.0000 - mae: 64470.5156 - val_loss: 9364030464.0000 - val_mae: 63920.2266
Epoch 3/50
18/18 [==============================] - 0s 2ms/step - loss: 9438983168.0000 - mae: 63064.7578 - val_loss: 9049596928.0000 - val_mae: 62073.9492
Epoch 4/50
18/18 [==============================] - 0s 5ms/step - loss: 9060588544.0000 - mae: 61191.2852 - val_loss: 8580209152.0000 - val_mae: 59358.7734
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 8525004288.0000 - mae: 58632.0781 - val_loss: 7973162496.0000 - val_mae: 56014.1133
Epoch 6/50
18/18 [==============================] - 0s 2ms/step - loss: 7884240384.0000 - mae: 55641.2227 - val_loss: 7239305728.0000 - val_mae: 52146.9961
Epoch 7/50
18/18 [==============================] - 0s 2ms/step - loss: 71

In [14]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Mean Absolute Error: {test_mae:.2f}")


5/5 [==============================] - 0s 1ms/step - loss: 5315935744.0000 - mae: 54961.1094
Test Mean Absolute Error: 54961.11


In [16]:
# Save the trained model to a file
model.save('gemstone_price_prediction_model_FNN.h5')


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the saved model
loaded_model = keras.models.load_model('gemstone_price_prediction_model_FNN.h5')

# Function to preprocess user input
def preprocess_input(user_input):
    # Extract user input
    date = pd.to_datetime(user_input['date'])
    gemstone_name = user_input['gemstone_name']
    color = user_input['color']
    clarity = user_input['clarity']
    weight = float(user_input['weight'])  # Ensure 'weight' is converted to a float

    # Create a DataFrame for prediction
    new_data = pd.DataFrame({
        'Date': [date],
        'GemstoneName': [gemstone_name],
        'Color': [color],
        'Clarity': [clarity],
        'Cut': [0],  # Since 'Cut' was not included in user input, you can assign a default value or handle it differently
        'Weight': [weight]
    })

    # Normalize the numerical feature
    new_data[['Weight']] = scaler.transform(new_data[['Weight']])

    return new_data

# User input
user_input = {
    'date': input('Enter the date (e.g., 2023-10-29): '),
    'gemstone_name': input('Enter the gemstone name: '),
    'color': input('Enter the color: '),
    'clarity': input('Enter the clarity: '),
    'weight': input('Enter the weight: ')
}

# Preprocess user input
new_data = preprocess_input(user_input)

# Make a price prediction
predicted_price = loaded_model.predict(new_data)
print(f"Predicted Price: ${predicted_price[0][0]:.2f}")


FileNotFoundError: [Errno 2] No such file or directory: 'gemstone_name_label_encoder.pkl'